In [32]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torchvision import models,datasets

# 12 使用Tensorboard在 PyTorch 中进行可视化
## Tensorboard 简介
Tensorboard是tensorflow内置的一个可视化工具，它通过将tensorflow程序输出的日志文件的信息可视化使得tensorflow程序的理解、调试和优化更加简单高效。 Tensorboard的可视化依赖于tensorflow程序运行输出的日志文件，因而tensorboard和tensorflow程序在不同的进程中运行。 TensorBoard给我们提供了极其方便而强大的可视化环境。它可以帮助我们理解整个神经网络的学习过程、数据的分布、性能瓶颈等等。

tensorboard虽然是tensorflow内置的可视化工具，但是他们跑在不同的进程中，所以Github上已经有大神将tensorboard应用到Pytorch中 链接在这里

## Tensorboard 安装
首先需要安装tensorboard

```pip install tensorboard```

安装完成后与 visdom一样执行独立的命令 ```tensorboard --logdir logs``` 即可启动，默认的端口是 6006,在浏览器中打开`http://localhost:6006/`即可看到web页面。

## 页面
与visdom不同，tensorboard针对不同的类型人为的区分多个标签，每一个标签页面代表不同的类型。 下面我们根据不同的页面功能做个简单的介绍，更多详细内容请参考官网

## SCALAR
对标量数据进行汇总和记录，通常用来可视化训练过程中随着迭代次数**准确率(val acc)**、**损失值(train/test loss)**、**学习率(learning rate)**、**每一层的权重和偏置的统计量(mean、std、max/min)**等的变化曲线

## IMAGES
可视化当前轮训练使用的训练/测试图片或者 feature maps

## GRAPHS
可视化计算图的结构及计算图上的信息，通常用来展示网络的结构

## HISTOGRAMS
可视化张量的取值分布，记录变量的直方图(统计张量随着迭代轮数的变化情况）

## PROJECTOR
全称Embedding Projector 高维向量进行可视化

## 使用
在使用前请先去确认执行tensorboard --logdir logs 并保证 http://localhost:6006/ 页面能够正常打开

### 图像展示
首先介绍比较简单的功能，查看我们训练集和数据集中的图像，这里我们使用现成的图像作为展示。这里使用wikipedia上的一张猫的图片这里

引入 tensorboardX 包


In [33]:

# 这里的引用也要修改成torch的引用
#from tensorboardX import SummaryWriter
from torch.utils.tensorboard import SummaryWriter

In [34]:
cat_img = Image.open('data/0a0c223352985ec154fd604d7ddceabd.jpg')
cat_img.size

(500, 375)


这是一张500x375的图，我们先把她变成224x224的图片，因为后面要使用的是vgg16

In [35]:
transform_224 = transforms.Compose([
        transforms.Resize(224), # 这里要说明下 Scale 已经过期了，使用Resize
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
cat_img_224=transform_224(cat_img)

将图片展示在tebsorboard中：

In [36]:
writer = SummaryWriter(log_dir='./logs', comment='cat image') # 这里的logs要与--logdir的位置一样
writer.add_image("cat",cat_img_224)
writer.close()# 执行close立即刷新，否则将每120秒自动刷新

浏览器访问 http://localhost:6006/#images 即可看到猫的图片

更新损失函数
更新损失函数和训练批次我们与visdom一样使用模拟展示，这里用到的是tensorboard的SCALAR页面

In [37]:
x = torch.FloatTensor([100])
y = torch.FloatTensor([500])

for epoch in range(30):
    x = x * 1.2
    y = y / 1.1
    loss = np.random.random()
    with SummaryWriter(log_dir='./logs', comment='train') as writer: #可以直接使用python的with语法，自动调用close方法
        writer.add_histogram('his/x', x, epoch)
        writer.add_histogram('his/y', y, epoch)
        writer.add_scalar('data/x', x, epoch)
        writer.add_scalar('data/y', y, epoch)
        writer.add_scalar('data/loss', loss, epoch)
        writer.add_scalars('data/data_group', {'x': x, 'y': y}, epoch)

浏览器访问 http://localhost:6006/#scalars 即可看到图形

## 使用PROJECTOR对高维向量可视化
PROJECTOR的的原理是通过PCA，T-SNE等方法将高维向量投影到三维坐标系（降维度）。Embedding Projector从模型运行过程中保存的checkpoint文件中读取数据，默认使用主成分分析法（PCA）将高维数据投影到3D空间中，也可以通过设置设置选择T-SNE投影方法，这里做一个简单的展示。

我们还是用第三章的mnist代码

In [38]:
BATCH_SIZE=512 
EPOCHS=20 
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out
model = ConvNet()
optimizer = torch.optim.Adam(model.parameters())
def train(model, train_loader, optimizer, epoch):
    n_iter=0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            n_iter=n_iter+1
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            #相对于以前的训练方法 主要增加了以下内容
            out = torch.cat((output.data, torch.ones(len(output), 1)), 1) # 因为是投影到3D的空间，所以我们只需要3个维度
            with SummaryWriter(log_dir='./logs', comment='mnist') as writer: 
                #使用add_embedding方法进行可视化展示
                writer.add_embedding(
                    out,
                    metadata=target.data,
                    label_img=data.data,
                    global_step=n_iter)


这里节省时间，只训练一次

In [39]:
train(model, train_loader, optimizer, 0)

Train Epoch: 0 [14848/60000 (25%)]	Loss: 0.296480
Train Epoch: 0 [30208/60000 (50%)]	Loss: 0.216185
Train Epoch: 0 [45568/60000 (75%)]	Loss: 0.157513


打开 http://localhost:6006/#projector 即可看到效果。

目前测试投影这部分也是有问题的，根据官网文档的代码进行测试，也显示不出来，正在找原因

绘制网络结构
在pytorch中我们可以使用print直接打印出网络的结构，但是这种方法可视化效果不好，这里使用tensorboard的GRAPHS来实现网络结构的可视化。 由于pytorch使用的是动态图计算，所以我们这里要手动进行一次前向的传播.

使用Pytorch已经构建好的模型进行展示

In [40]:
vgg16 = models.vgg16(pretrained=True) # 这里下载预训练好的模型
print(vgg16) # 打印一下这个模型

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/shawn/.cache/torch/hub/checkpoints/vgg16-397923af.pth
34.4%

在前向传播前，先要把图片做一些调整

In [ ]:
transform_2 = transforms.Compose([
    transforms.Resize(224), 
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

使用上一张猫的图片进行前向传播

In [ ]:
vgg16_input=transform_2(cat_img)[np.newaxis]# 因为pytorch的是分批次进行的，所以我们这里建立一个批次为1的数据集
vgg16_input.shape

torch.Size([1, 3, 224, 224])

开始前向传播，打印输出值

In [ ]:
out = vgg16(vgg16_input)
_, preds = torch.max(out.data, 1)
label=preds.numpy()[0]
label

463

将结构图在tensorboard进行展示

In [ ]:
with SummaryWriter(log_dir='./logs', comment='vgg161') as writer:
    writer.add_graph(resnet50, vgg16_input)